## Imports and Loads

In [1]:
from base import *
from scipy import stats as st
from services import *
from functools import reduce


In [2]:
ATLANTICO_NORTE = AtlanticoNorte()
ATLANTICO_SUL = AtlanticoSul()
PACIFICO_SUL = PacificoSul()
NINO = Nino()
COORDENADAS = Coordenadas()
PRECIPITACAO = Precipitacao('R')
PREPROCESSAMENTO = PreProcessing()
#PRECIPITACAO.encoder()

In [4]:
PATH_DADOS_DEFASADOS_OUTPUT="content/dados_defasados.xlsx"
PATH_TABELA_CORRELACOES="content/tabela_de_correlacoes.xlsx"
PATH_VARIAVEIS_SELECIONADAS="content/variaveis_selecionadas_por_area.csv"
PATH_BASE_DE_SUBMISSAO="content/base_treino_teste.csv"

## Defasando os dados

In [5]:
atlantico_norte_deslocado = PREPROCESSAMENTO.gerar_base_de_dados_deslocada(ATLANTICO_NORTE.data, "TSM-AN")
atlantico_sul_deslocado = PREPROCESSAMENTO.gerar_base_de_dados_deslocada(ATLANTICO_SUL.data, "TSM-AS")
pacifico_sul_deslocado = PREPROCESSAMENTO.gerar_base_de_dados_deslocada(PACIFICO_SUL.data, "TSM-PS")
nino_deslocado = PREPROCESSAMENTO.gerar_base_de_dados_deslocada(NINO.data, "TSM-NI")

keys = nino_deslocado.keys()

In [20]:
base = {}
for r in keys:
    concatenate = [atlantico_norte_deslocado[r], atlantico_sul_deslocado[r], pacifico_sul_deslocado[r], nino_deslocado[r]]
    base[r] = pd.concat(concatenate, axis=1)

data = []
for k in base.keys():
    data.append(base[k])

pd.concat(data, axis=1).to_excel(PATH_DADOS_DEFASADOS_OUTPUT)

## Geração da Tabela de Correlações

In [7]:
defasagens = pd.read_excel(PATH_DADOS_DEFASADOS_OUTPUT, index_col=[0])
cor = {}
for p in PRECIPITACAO.data.columns:
    var = []
    for col in defasagens.columns:
        i = col
        j = 'PRM'+p

        a = pd.merge(defasagens[col],PRECIPITACAO.data[p], on=['Data'], how='outer').dropna()
        a.columns=[i,j]
        var.append(st.spearmanr(a[i],a[j])[0])
    cor['PRM'+p] = var
        

pd.DataFrame(cor).set_index(defasagens.columns).T.to_excel(PATH_TABELA_CORRELACOES)

## Seleção de Variáveis

In [21]:
tabela_correlacoes = pd.read_excel(PATH_TABELA_CORRELACOES, index_col=[0])


variaveis_selecionadas = {}
for r in tabela_correlacoes.T.columns:
    data_abs = abs(tabela_correlacoes.T)
    indices = list(data_abs.sort_values(by=[str(r)], ascending=False)[r].head(5).index)
    variaveis_selecionadas[r] = indices

nvars = pd.DataFrame(variaveis_selecionadas)
nvars.to_csv(PATH_VARIAVEIS_SELECIONADAS)
nvars

,PRM1,PRM2,PRM3,PRM4,PRM5,PRM6,PRM7,PRM8,PRM9,PRM10,...,PRM15,PRM16,PRM17,PRM18,PRM19,PRM20,PRM21,PRM22,PRM23,PRM24
0,TSM-PS12(5),TSM-PS12(5),TSM-PS12(5),TSM-AS27(5),TSM-AS27(5),TSM-AS27(5),TSM-AS27(5),TSM-AS27(5),TSM-AS27(5),TSM-AS27(5),...,TSM-AS39(5),TSM-AS27(5),TSM-AS27(5),TSM-AS27(5),TSM-AS30(5),TSM-AS37(5),TSM-AS35(5),TSM-AS5(11),TSM-AS27(5),TSM-AS27(5)
1,TSM-AS10(11),TSM-AS34(6),TSM-AS19(11),TSM-AS18(11),TSM-AS18(11),TSM-AS35(5),TSM-AS11(11),TSM-AS33(5),TSM-AS18(11),TSM-AS18(11),...,TSM-AS27(5),TSM-AS33(5),TSM-AS33(5),TSM-AS33(5),TSM-AS37(5),TSM-AS30(5),TSM-PS32(5),TSM-AS27(5),TSM-AS11(11),TSM-AS33(5)
2,TSM-AS34(6),TSM-AS20(11),TSM-AS34(6),TSM-AS19(11),TSM-AS17(11),TSM-AS18(11),TSM-PS33(5),TSM-AS35(5),TSM-AS33(5),TSM-AS17(11),...,TSM-AS33(5),TSM-AS39(5),TSM-AS11(11),TSM-AS39(5),TSM-AS31(5),TSM-AS5(11),TSM-AS37(5),TSM-AS33(5),TSM-AS33(5),TSM-AS39(5)
3,TSM-PS20(6),TSM-AS10(11),TSM-AS27(5),TSM-PS12(5),TSM-AS35(5),TSM-AS11(11),TSM-PS27(5),TSM-PS33(5),TSM-AS39(5),TSM-AS35(5),...,TSM-AS35(5),TSM-AS18(11),TSM-AS39(5),TSM-AS11(11),TSM-AS5(11),TSM-AS36(5),TSM-AS39(5),TSM-AS39(5),TSM-AS5(11),TSM-AS11(11)
4,TSM-PS19(5),TSM-AS19(11),TSM-AS20(11),TSM-AS11(11),TSM-AS12(11),TSM-AS17(11),TSM-AS33(5),TSM-AS18(11),TSM-AS35(5),TSM-AS33(5),...,TSM-AS18(11),TSM-AS5(11),TSM-AS35(5),TSM-AS35(5),TSM-AS29(5),TSM-AS35(5),TSM-AS36(5),TSM-AS11(11),TSM-AS39(5),TSM-AS5(11)


In [22]:
defasagens = pd.read_excel(PATH_DADOS_DEFASADOS_OUTPUT, index_col=[0])
sel = {}
for v in nvars.columns:
    sel[v] = []
    for r in nvars[v]:
        if r in defasagens.columns:
            sel[v].append(defasagens[r]) 

In [37]:
areas = ["PRM1","PRM2","PRM3","PRM4","PRM5","PRM6","PRM7","PRM8","PRM9","PRM10","PRM11",
                  "PRM12","PRM13","PRM14","PRM15","PRM16","PRM17","PRM18","PRM19","PRM20","PRM21",
                  "PRM22","PRM23","PRM24"]

PRECIPITACAO.data.columns = areas
data = []
for area in sel.keys():
    prm = reduce(lambda left, right: pd.merge(left,right, on=['Data'], how='outer'),sel[area])
    data.append(prm)
data = reduce(lambda left, right: pd.merge(left,right, on=['Data'], how='outer', suffixes=('', '_drop')),data)

prm = pd.merge(data,PRECIPITACAO.data[areas], on=['Data'], how='outer')

prm = prm[prm.columns.drop(list(prm.filter(regex='_drop')))]
prm.dropna().to_csv(PATH_BASE_DE_SUBMISSAO)

In [38]:
import pandas as pd

data = pd.read_csv(PATH_BASE_DE_SUBMISSAO)
data = data.drop(columns=['Data'])
data

,TSM-PS12(5),TSM-AS10(11),TSM-AS34(6),TSM-PS20(6),TSM-PS19(5),TSM-AS20(11),TSM-AS19(11),TSM-AS27(5),TSM-AS18(11),TSM-AS11(11),...,PRM15,PRM16,PRM17,PRM18,PRM19,PRM20,PRM21,PRM22,PRM23,PRM24
0,20.5793,25.9649,19.7510,19.3091,20.0670,22.6626,23.7538,18.7158,25.3597,26.7906,...,174.0490,173.5420,172.5370,160.1850,236.46100,196.9080,188.2280,186.2690,175.8200,155.40700
1,19.9703,26.8902,18.4993,18.2039,19.6867,23.6206,24.6150,17.6094,25.8847,27.6365,...,240.4790,258.0340,268.0700,309.3150,185.63900,193.9470,239.9810,267.3700,281.1310,329.37300
2,20.0107,27.6941,18.2785,17.7718,19.8742,24.1323,25.1708,17.9266,26.3987,28.1135,...,221.2850,208.8750,235.0530,260.7010,280.96900,264.6620,213.3500,194.4960,202.2550,234.70100
3,20.5849,27.4212,18.8111,18.1245,20.3675,23.6960,24.8512,18.4656,26.2031,27.5871,...,305.2530,306.7140,317.6520,324.1510,248.74100,269.1220,278.7870,298.8590,301.0820,304.18800
4,21.4757,26.6367,18.7258,18.5417,20.4125,22.8948,24.3282,18.9680,25.4828,25.9994,...,87.3727,79.3388,71.0448,56.8430,95.87570,91.2394,84.2987,85.2029,76.3257,64.96080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,26.4304,23.1263,23.2349,24.4778,23.6661,20.0047,21.8937,23.2232,23.5927,23.0543,...,23.6752,23.0330,14.9606,10.8813,6.95271,15.4828,21.7423,27.8090,17.1233,7.22894
452,26.0217,23.4339,22.9988,24.4513,23.5494,20.1980,21.8546,21.8648,23.6179,23.8980,...,21.4387,41.4355,91.1224,51.2656,48.36010,30.9106,14.6627,19.9980,29.8156,13.02310
453,23.8826,24.6634,21.8662,23.1672,21.9253,21.1668,23.0587,21.2401,25.3117,25.9916,...,48.6008,54.1594,57.0661,74.2762,109.08500,67.8707,45.8699,55.5058,50.6319,41.58150
454,22.1416,25.8891,20.2062,20.8033,20.8847,22.3237,23.6579,20.0035,25.4081,27.4457,...,205.9550,208.5900,194.6640,214.9410,118.51100,177.2550,190.2710,215.2200,190.8810,166.06100


In [39]:
somas = []
for col in data.columns:
    c = []
    i = 0
    x = 3
    while x < 456:   
        c.append(sum(data[col][i:x]))
        i+=3
        x+=3 
        somas.append(c)

len(somas)

columns = data.columns
dc = {}

i = 0
for c in columns:
    dc[c] = somas[i]
    i+=1  

pd.DataFrame(dc).to_csv('content/base_de_dados_abragencia_3_meses.csv')


In [40]:
data

,TSM-PS12(5),TSM-AS10(11),TSM-AS34(6),TSM-PS20(6),TSM-PS19(5),TSM-AS20(11),TSM-AS19(11),TSM-AS27(5),TSM-AS18(11),TSM-AS11(11),...,PRM15,PRM16,PRM17,PRM18,PRM19,PRM20,PRM21,PRM22,PRM23,PRM24
0,20.5793,25.9649,19.7510,19.3091,20.0670,22.6626,23.7538,18.7158,25.3597,26.7906,...,174.0490,173.5420,172.5370,160.1850,236.46100,196.9080,188.2280,186.2690,175.8200,155.40700
1,19.9703,26.8902,18.4993,18.2039,19.6867,23.6206,24.6150,17.6094,25.8847,27.6365,...,240.4790,258.0340,268.0700,309.3150,185.63900,193.9470,239.9810,267.3700,281.1310,329.37300
2,20.0107,27.6941,18.2785,17.7718,19.8742,24.1323,25.1708,17.9266,26.3987,28.1135,...,221.2850,208.8750,235.0530,260.7010,280.96900,264.6620,213.3500,194.4960,202.2550,234.70100
3,20.5849,27.4212,18.8111,18.1245,20.3675,23.6960,24.8512,18.4656,26.2031,27.5871,...,305.2530,306.7140,317.6520,324.1510,248.74100,269.1220,278.7870,298.8590,301.0820,304.18800
4,21.4757,26.6367,18.7258,18.5417,20.4125,22.8948,24.3282,18.9680,25.4828,25.9994,...,87.3727,79.3388,71.0448,56.8430,95.87570,91.2394,84.2987,85.2029,76.3257,64.96080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,26.4304,23.1263,23.2349,24.4778,23.6661,20.0047,21.8937,23.2232,23.5927,23.0543,...,23.6752,23.0330,14.9606,10.8813,6.95271,15.4828,21.7423,27.8090,17.1233,7.22894
452,26.0217,23.4339,22.9988,24.4513,23.5494,20.1980,21.8546,21.8648,23.6179,23.8980,...,21.4387,41.4355,91.1224,51.2656,48.36010,30.9106,14.6627,19.9980,29.8156,13.02310
453,23.8826,24.6634,21.8662,23.1672,21.9253,21.1668,23.0587,21.2401,25.3117,25.9916,...,48.6008,54.1594,57.0661,74.2762,109.08500,67.8707,45.8699,55.5058,50.6319,41.58150
454,22.1416,25.8891,20.2062,20.8033,20.8847,22.3237,23.6579,20.0035,25.4081,27.4457,...,205.9550,208.5900,194.6640,214.9410,118.51100,177.2550,190.2710,215.2200,190.8810,166.06100
